## Certify and install required python modules.

In [ ]:
import os
import sys
import subprocess
import pkg_resources

__require = {'casatools', 'casatasks', 'casatestutils', 'panel', 'astropy', 'matplotlib', 'numpy'}
__installed = {pkg.key for pkg in pkg_resources.working_set}
    
__missing = __require - __installed

os.chdir('../')

if len(__missing) > 0:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *__missing], stdout=subprocess.DEVNULL)

In [ ]:
!python3 -m casatools --update-user-data

## Utility Functions

In [ ]:
def clean_data(prefix=''):
    import os
    import glob
    import shutil
    
    for file in glob.glob('{0}/{1}*.*'.format(os.getcwd(), prefix)):
        print(os.getcwd())
        try:
            shutil.rmtree(file)
        except Exception as error:
            print('Failed to remove {0}: {1}'.format(file, error))

## Stakeholder Dataset

```
Dataset(s) (MOUS)
E2E6.1.00034.S (uid://A002/Xcff05c/X1ec)

Test list:
1c.  SF cube with pcwdT, weighting=briggsbwtaper - E2E6.1.00034.S

Each test stores reference values in dictionaries for the metrics
to be tested and these dictionaries are stored in a single nested dictionary
in a json file located in the casatestdata repository. 
The path of json file is stored in the variable, 
       self.expdict_jsonfile  
in test_tclean_base.setUp(). 

* NOTE for updating the tests and fiducial values in json file *
When the json file is updated and its 'casa_version'
could also be updated then self.refversion in the setUp() needs to be updated to
match with the 'casa_version' as defined in the json file otherwise 
almastkteestutils.read_testcase_expdicts() print an error message.

The fudicial metric values for a specific image are stored with the following keys.
 
For the standard tests, default sets are:
    exp_im_stats, exp_mask_stats, exp_pb_stats, exp_psf_stats,
    exp_model_stats, exp_resid_stats, exp_sumwt_stats
For mosaic tests, the ones above and
    exp_wt_stats (for mosaic)
Additionally, for cube imaging (if self.parallel=True),
    exp_bmin_dict, exp_bmaj_dict, exp_pa_dict
And for mtmfs
    exp_im1_stats, exp_model1_stats, exp_resid1_stats, exp_sumwt1_stats
```

## Load Stakeholder Data

In [ ]:
os.mkdir('data/')

!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://www.cv.nrao.edu/~jhoskins/E2E6.1.00034.S_tclean.ms.tar
!tar -xvf E2E6.1.00034.S_tclean.ms.tar
os.system('mv E2E6.1.00034.S_tclean.ms/ data/E2E6.1.00034.S_tclean.ms/')

!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://www.cv.nrao.edu/~jhoskins/test_stk_alma_pipeline_imaging_exp_dicts.json
os.system('cp test_stk_alma_pipeline_imaging_exp_dicts.json ./data/')    

## Import Required Dependencies

In [ ]:
import casatasks
import casatools
import panel as pn

from astropy.io import fits
from astropy.wcs import WCS

import matplotlib.pyplot as plt
import numpy as np

import tests.alma.test_standard_cube_briggsbwtaper as stk

## User Stakeholder Test

The unit test script is broken into **three** core parts within the notebook.

- Setup
- Modifiable Tests
- Metric Checks and Reporting

### Setup 
The setup section handles setting up the untit test and general class instantiation. In general the user shouldn't have to change this. For those that are developing their own stakeholder test, the setup section would include all the functions required to run the unit tests along with any user defined utility functions specific to their tests.

### Testing
The testing section makes the stakeholder test code available to the user such that they can modify parameters, check their diagnostic and rerun the tests. The tests in the notebook are presented as they are defined in the unit testing script including their default values. **It is recommended that the user restart the kernel and run all after each change**

### Metric Checks and Testing
This section runs the unit tests built in metric code as well as allowing the user to write their won diagnostics code. In addition, the user can access and tweak the values in the 'expected metrics' JSON using the setter/getter functionality.

**Getting**
`new_dict = standard.exp_dict`

**Modifying**
`new_dict['exp_im_stats']['im_rms'][1] = <new value>`

**Setting**
`standard.exp_dict = new_dict`

The user can now rerun the metric test functions and the new dictionay will be used. This will not be the case if the kernel or the testing class is reinstantiated.

## Setup

In [ ]:
standard = stk.Test_standard()

standard.set_file_path(path=os.getcwd()+"/data/")

standard.setUp()
standard.test_standard_cube_briggsbwtaper()

## Testing

The head(footer) are used with `nbsyc.py` which allows the user to sync change either from `notebook` --> `scipt` or `script` --> `notebook`. The formatting of the headers(footers) is important. There must be a single whitespace after the header and before the footer. Any changes made between the header(footer) will be sync with `nbsync.py`.

In [ ]:
msfile = standard.data_path + 'E2E6.1.00034.S_tclean.ms'
file_name = standard.file_name

# %% test_standard_cube_briggsbwtaper_tclean_1 start @

casatasks.tclean(vis=msfile, 
                 imagename=file_name+'0', 
                 field='1',
                 spw=['0'], 
                 imsize=[80, 80], 
                 antenna=['0,1,2,3,4,5,6,7,8'], 
                 scan=['8,12,16'], 
                 intent='OBSERVE_TARGET#ON_SOURCE',
                 datacolumn='data', 
                 cell=['1.1arcsec'], 
                 phasecenter='ICRS 00:45:54.3836 -073.15.29.413', 
                 stokes='I', 
                 specmode='cube',
                 nchan=508, 
                 start='220.2526743594GHz', 
                 width='0.2441741MHz',
                 outframe='LSRK', 
                 pblimit=0.2, 
                 perchanweightdensity=True,
                 gridder='standard', 
                 mosweight=False,
                 deconvolver='hogbom', 
                 usepointing=False, 
                 restoration=False,
                 pbcor=False, 
                 weighting='briggsbwtaper', 
                 restoringbeam='common',
                 robust=0.5, npixels=0, 
                 niter=0, 
                 threshold='0.0mJy', 
                 nsigma=0.0,
                 interactive=0, 
                 usemask='auto-multithresh',
                 sidelobethreshold=1.25, 
                 noisethreshold=5.0,
                 lownoisethreshold=2.0, 
                 negativethreshold=0.0, 
                 minbeamfrac=0.1,
                 growiterations=75, 
                 dogrowprune=True, 
                 minpercentchange=1.0,
                 fastnoise=False, 
                 savemodel='none', 
                 parallel=False,
                 verbose=True)

# %% test_standard_cube_briggsbwtaper_tclean_1 end @


In [ ]:
print('Copying iter0 files to iter1')
standard.copy_products(file_name+'0', file_name+'1')

casatasks.imstat(standard.file_name + '0.psf')

In [ ]:
# %% test_standard_cube_briggsbwtaper_tclean_2 start @

casatasks.tclean(vis=msfile, 
                 imagename=file_name+'1', 
                 field='1',
                 spw=['0'], 
                 imsize=[80, 80], 
                 antenna=['0,1,2,3,4,5,6,7,8'],
                 scan=['8,12,16'], 
                 intent='OBSERVE_TARGET#ON_SOURCE',
                 datacolumn='data', 
                 cell=['1.1arcsec'], 
                 phasecenter='ICRS 00:45:54.3836 -073.15.29.413', 
                 stokes='I', 
                 specmode='cube',
                 nchan=508, 
                 start='220.2526743594GHz', 
                 width='0.2441741MHz',
                 outframe='LSRK', 
                 perchanweightdensity=True,
                 usepointing=False, 
                 pblimit=0.2, 
                 nsigma=0.0,
                 gridder='standard', 
                 mosweight=False, 
                 deconvolver='hogbom', 
                 restoration=True, 
                 restoringbeam='common', 
                 pbcor=True, 
                 weighting='briggsbwtaper', 
                 robust=0.5, 
                 npixels=0, 
                 niter=20000,
                 threshold='0.354Jy', 
                 interactive=0, 
                 usemask='auto-multithresh', 
                 sidelobethreshold=1.25, 
                 noisethreshold=5.0, 
                 lownoisethreshold=2.0, 
                 negativethreshold=0.0,
                 minbeamfrac=0.08, 
                 growiterations=75, 
                 dogrowprune=True,
                 minpercentchange=1.0, 
                 fastnoise=False, 
                 restart=True, 
                 calcres=False, 
                 calcpsf=False, 
                 savemodel='none',
                 parallel=False, 
                 verbose=True)

# %% test_standard_cube_briggsbwtaper_tclean_2 end @

## Produce Standard Cube Report

In [ ]:
standard.standard_cube_report()

### Display Weblog

In [ ]:
from IPython.display import IFrame

IFrame(src='../test_tclean_alma_pipeline_weblog.html', width=900, height=600)

## Clean Data
Uncomment this if you want to clean the working files out of the directory.

In [ ]:
clean_data(prefix='standard_cube')

## Example: Modifying Expected Metrics JSON

In [ ]:
# Getting expected values JSON

import json

new_dict = standard.exp_dict
new_dict_pretty = json.dumps(new_dict, indent=4)

print(new_dict_pretty)

In [ ]:
# Modifying expected values JSON

new_dict['exp_im_stats']['com_bmin'][1] = 6.203

In [ ]:
# Setting expected values JSON

standard.exp_dict = new_dict

exp_dict_pretty = json.dumps(standard.exp_dict, indent=4)

print(exp_dict_pretty)